#### Setup

In [1]:
# Import dataset class and accuracy functions
from __init__ import PricingWizardDataset, regression_accuracy, threshold_accuracy, base_regression_pipeline, load_model

# Model loading imports
import joblib
import torch

# Import other libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Others...

In [28]:
# Load Data
data = PricingWizardDataset()

Dataset Loaded: post_preprocessing_without_dummies
	Number of Rows: 283055
	Number of Columns: 22
	Outlier Removal: True
	Train Size: 0.8
	Test Size: 0.2
	Random State: 42


In [29]:
# Current working directory
cwd = os.getcwd()

# model directory
model_dir = os.path.join(cwd, 'models', 'pickled_models')

In [ ]:
# 
def get_listing_prices(classified_ids):
    # Extract listing prices from unprocessed dataset
    listing_prices = data.raw_df[data.raw_df.classified_id.isin(classified_ids)].listing_price.to_numpy()

#### Base Regression Model

In [30]:
# Perform preprocessing
data.apply_function(base_regression_pipeline)


# Load model
model = load_model(f'{model_dir}/base_regression.pkl')

# Split
_, X_test, _, y_test = data.stratify_train_test_split(val_size=0,
                                                      y_column='log_listing_price')

Model loaded successfully from /Users/rasmuskrebs/Documents/School/semester_3/data_mining/pricing_wizards/models/pickled_models/base_regression.pkl
Dependent variable distribution is equal across all subsets


In [39]:
# Predict
y_pred = model.predict(X_test)

# Calculate accuracy
print('Log Scale Accuracy')
regression_accuracy(y_pred, y_test)
threshold_accuracy(y_pred, y_test, p=0.1)

print('\nOriginal Scale Accuracy')
regression_accuracy(np.exp(y_pred), np.exp(y_test))
threshold_accuracy(np.exp(y_pred), np.exp(y_test), p=0.1)

Log Scale Accuracy
R2 Score: 0.5741421546170012
MSE: 0.41030475099260805
MAE 0.4958842894677665
RMSE 0.6405503500839009
Threshold Accuracy 0.6503683029799863

Original Scale Accuracy
R2 Score: 0.36220203471317636
MSE: 189674.0987414045
MAE 207.70095452684768
RMSE 435.5158995276803
Threshold Accuracy 0.13370193072017805
